# Tutorials i TensorFlow fra Kieth Downing
## Sett 3, General Artificial Neural Network (GANN)

Hele denne strukturen er objektorientert. Gann består av tre klasser:

1. Gann	(The main network with all configurations).
2. GannModule (A layer in a neural net)
3. Caseman (Case manager)

Denne gjennomgangen går igjennom de forskjellige klassene, metode for metode.


In [1]:
import math
import matplotlib.pyplot as PLT
import numpy as np
import tensorflow as tf
from downing_code import tflowtools as TFT

## Gann(): General Artificial Neural Network
I ``__init__()`` metoden er det hovedsakelig kun parametere som blir satt. Alle parametere er enten for output eller for å bygge nettet. Metoden ender med et kall på __"build()"__ som bygger opp selve nettet.

In [2]:
def __init__(self, dims, cman,lrate=.1,showint=None,mbs=10,vint=None,softmax=False):
        self.learning_rate = lrate
        self.layer_sizes = dims # Sizes of each layer of neurons
        self.show_interval = showint # Frequency of showing grabbed variables
        self.global_training_step = 0 # Enables coherent data-storage during extra training runs (see runmore).
        self.grabvars = []  # Variables to be monitored (by gann code) during a run.
        self.grabvar_figures = [] # One matplotlib figure for each grabvar
        self.minibatch_size = mbs
        self.validation_interval = vint
        self.validation_history = []
        self.caseman = cman
        self.softmax_outputs = softmax
        self.modules = []
        self.build()

In [5]:
def build(self):
    tf.reset_default_graph()  # This is essential for doing multiple runs!!
    num_inputs = self.layer_sizes[0]
    self.input = tf.placeholder(tf.float64, shape=(None, num_inputs), name='Input')
    
    invar = self.input
    insize = num_inputs
    
    # Build all of the modules
    for i,outsize in enumerate(self.layer_sizes[1:]):
        gmod = Gannmodule(self,i,invar,insize,outsize)
        invar = gmod.output
        insize = gmod.outsize
        
    self.output = gmod.output # Output of last module is output of whole network
    if self.softmax_outputs: self.output = tf.nn.softmax(self.output)
    self.target = tf.placeholder(tf.float64,shape=(None,gmod.outsize),name='Target')
    self.configure_learning()

Nettet bygges opp i ``build()`` metoden vist under. Denne metoden er ment til å være generalisert for å kunne lage et nett av ønskede dimensjoner. Dimensjonene kommer da fra ``__init__()`` metoden.

Input er en tf.placeholder() som vanlig. Det som er interressant med den er at den er 2-dimensjonal. Den holder på en ``shape=(None, num_inputs)``. Når en av dimensjonene i shape er satt til å være "None" betyr dette at den er av ukjent dimensjon. Den kan da altså ha en ukjent mengde rader.

#### Digresjon: Litt om Gann Module
![modell](https://github.com/MagnusPoppe/Tensorflow-Interface/blob/master/GannModule.png?raw=true)

GannModule er et lag i nettet. Dette er definert med at det har vektene som kommer før seg selv og utverdien (som blir selve laget). Den kjenner også til innverdien som kommer før vektene. 

Fokuset på Gann modulen er dermed selve vekt-laget. Den kjenner også til hva som kommer inn- og hva som kommer ut. Det som kommer ut er definert med 2 verdier, utverdien (en operator) og størrelsen på output

En løkke for å bygge opp de forskjellige modulene (lagene) i nettet kommer så:

In [7]:
invar = self.input
insize = num_inputs

# Build all of the modules
for i,outsize in enumerate(self.layer_sizes[1:]):
    gmod = Gannmodule(self,i,invar,insize,outsize)
    invar = gmod.output
    insize = gmod.outsize

NameError: name 'self' is not defined

Før løkken defineres de initielle verdiene som trengs ved første kjøring. Disse er da følgende: 
- __invar__: En inn-variabel. For GannModule blir dette forrige lag, eller da laget før vektmatrisen.
- __insize__: Dimensjonen på innvariabelen. Denne kan da kun være 1dimensjonal. 

Løkken kjører så, og lager modul for modul (eller lag for lag). For hver runde i løkken lages først laget, så erstattes invar og inside med det som ble output av det forrige laget. Dette er hele oppbyggingen av hvert lag.

Etter løkken lagres den endelige output modulen. Hvis soft-max outputs er ønsket blir disse også lagret. Læringen blir så konfigurert. 


In [8]:
def configure_learning(self):
    self.error = tf.reduce_mean(tf.square(self.target - self.output),name='MSE')
    self.predictor = self.output  # Simple prediction runs will request the value of output neurons
    
    # Defining the training operator
    optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
    self.trainer = optimizer.minimize(self.error,name='Backprop')

I configure learning settes errorfunksjonen og predictor (som brukes ved testing). Han definerer også optimeringsfunskjonen og lagrer dette.

# Trening - og Testingsfunkjsonen er lik som tidligere. Det resterende er for det meste kjøring av GANN og ikke noe man trenger gå igjennom her. 